In [ ]:
import cv2
import time
import math
import os
import numpy as np
import tensorflow as tf

import locality_aware_nms as nms_locality
import lanms

import model
FLAGS = tf.app.flags.FLAGS
#tf.app.flags.DEFINE_string('backbone', 'Mobilenet', 'what kind of backbone')
#tf.app.flags.DEFINE_string('dataset', 'icdar15', '')


tf.app.flags.DEFINE_string('f', '', 'kernel')
#FLAGS.backbone = 'Mobilenet' # Pvanet, Mobilenet, Resnet
#FLAGS.decoder = 'CRAFT' # original, CRAFT, Expand
#FLAGS.dataset = 'icdar2017_mlt'

from icdar import restore_rectangle
import time
from skimage import io
import matplotlib.pyplot as plt
from matplotlib.patches import Polygon
import colorsys
import random
from matplotlib import patches,  lines
%matplotlib inline


In [ ]:
#test_data_path = '/home/minjun/Jupyter/data/ocr/2015/test'
#test_data_path = '/home/minjun/Jupyter/data/ocr/2013/test/'
test_data_path = '/home/minjun/Jupyter/data/ocr/2017_MLT/test'
gpu_list = [2]
checkpoint_path = './tmp/east_icdar2017_mobilenet_v2_CRAFT/'
#checkpoint_path = './Untitled Folder/'
output_dir = './tmp/output'
no_write_images = True
dataset = 'icdar2017_mlt'


In [ ]:
def get_images():
    '''
    find image files in test data path
    :return: list of files found
    '''
    files = []
    exts = ['jpg', 'png', 'jpeg', 'JPG']
    for parent, dirnames, filenames in os.walk(test_data_path):
        for filename in filenames:
            for ext in exts:
                if filename.endswith(ext):
                    files.append(os.path.join(parent, filename))
                    break
    print('Find {} images'.format(len(files)))
    return files
def resize_image(im, max_side_len=2400):
    '''
    resize image to a size multiple of 32 which is required by the network
    :param im: the resized image
    :param max_side_len: limit of max image size to avoid out of memory in gpu
    :return: the resized image and the resize ratio
    '''
    h, w, _ = im.shape

    resize_w = w
    resize_h = h

    # limit the max side
    if max(resize_h, resize_w) > max_side_len:
        ratio = float(max_side_len) / resize_h if resize_h > resize_w else float(max_side_len) / resize_w
    else:
        ratio = 1.
    resize_h = int(resize_h * ratio)
    resize_w = int(resize_w * ratio)

    resize_h = resize_h if resize_h % 32 == 0 else (resize_h // 32 - 1) * 32
    resize_w = resize_w if resize_w % 32 == 0 else (resize_w // 32 - 1) * 32
    resize_h = max(32, resize_h)
    resize_w = max(32, resize_w)
    im = cv2.resize(im, (int(resize_w), int(resize_h)))

    ratio_h = resize_h / float(h)
    ratio_w = resize_w / float(w)

    return im, (ratio_h, ratio_w)




def detect(score_map, geo_map, timer, score_map_thresh=0.8, box_thresh=0.1, nms_thres=0.2):
    '''
    restore text boxes from score map and geo map
    :param score_map:
    :param geo_map:
    :param timer:
    :param score_map_thresh: threshhold for score map
    :param box_thresh: threshhold for boxes
    :param nms_thres: threshold for nms
    :return:
    '''
    if len(score_map.shape) == 4:
        score_map = score_map[0, :, :, 0]
        geo_map = geo_map[0, :, :, ]
    # filter the score map
    xy_text = np.argwhere(score_map > score_map_thresh)
    # sort the text boxes via the y axis
    xy_text = xy_text[np.argsort(xy_text[:, 0])]
    # restore
    start = time.time()
    text_box_restored = restore_rectangle(xy_text[:, ::-1]*4, geo_map[xy_text[:, 0], xy_text[:, 1], :]) # N*4*2
    #print('{} text boxes before nms'.format(text_box_restored.shape[0]))
    boxes = np.zeros((text_box_restored.shape[0], 9), dtype=np.float32)
    boxes[:, :8] = text_box_restored.reshape((-1, 8))
    boxes[:, 8] = score_map[xy_text[:, 0], xy_text[:, 1]]
    timer['restore'] = time.time() - start
    # nms part
    start = time.time()
    # boxes = nms_locality.nms_locality(boxes.astype(np.float64), nms_thres)
    boxes = lanms.merge_quadrangle_n9(boxes.astype('float32'), nms_thres)
    timer['nms'] = time.time() - start

    if boxes.shape[0] == 0:
        return None, timer

    # here we filter some low score boxes by the average score map, this is different from the orginal paper
    for i, box in enumerate(boxes):
        mask = np.zeros_like(score_map, dtype=np.uint8)
        cv2.fillPoly(mask, box[:8].reshape((-1, 4, 2)).astype(np.int32) // 4, 1)
        boxes[i, 8] = cv2.mean(score_map, mask)[0]
    boxes = boxes[boxes[:, 8] > box_thresh]
    print("def detect, box shape : ",np.shape(boxes))

    return boxes, timer

def simple_detect(score_map,rawshape, thres=0.5,thres_area= 5):
    print(np.shape(score_map))

    ret, binimage = cv2.threshold(score_map,thres,1,0)
    binimage = binimage.astype(np.int8)

    nlabels, labels, stats, centroids = cv2.connectedComponentsWithStats(binimage)
    boxes=[]
    print("box 갯수 : ",nlabels-1)
    for i in range(1,nlabels):
        area = stats[i, cv2.CC_STAT_AREA]
        center_x = int(centroids[i, 0])
        center_y = int(centroids[i, 1]) 
        left = stats[i, cv2.CC_STAT_LEFT]
        top = stats[i, cv2.CC_STAT_TOP]
        width = int(stats[i, cv2.CC_STAT_WIDTH]*1.2)
        height = int(stats[i, cv2.CC_STAT_HEIGHT]*1.2)

        y1 = max(min((center_y-height/2)/binimage.shape[0],1),0)
        y2 = max(min((center_y+height/2)/binimage.shape[0],1),0)
        x1 = max(min((center_x-width/2)/binimage.shape[1],1),0)
        x2 = max(min((center_x+width/2)/binimage.shape[1],1),0)

        boxes.append([[int(x1*rawshape[1]),int(y1*rawshape[0])],[int(x2*rawshape[1]),int(y2*rawshape[0])]])
    boxes = np.array(boxes)
    return boxes


def sort_poly(p):
    min_axis = np.argmin(np.sum(p, axis=1))
    p = p[[min_axis, (min_axis+1)%4, (min_axis+2)%4, (min_axis+3)%4]]
    if abs(p[0, 0] - p[1, 0]) > abs(p[0, 1] - p[1, 1]):
        return p
    else:
        return p[[0, 3, 2, 1]]
    
def random_colors(N, bright=True):
    """
    Generate random colors.
    To get visually distinct colors, generate them in HSV space then
    convert to RGB.
    """
    brightness = 1.0 if bright else 0.7
    hsv = [(i / N, 1, brightness) for i in range(N)]
    colors = list(map(lambda c: colorsys.hsv_to_rgb(*c), hsv))
    random.shuffle(colors)
    return colors
    
from matplotlib.patches import Polygon

def visualize(image, boxes,figsize=(16, 16)):
        """
        desc : bbox와 함께 이미지를 그린다.
        
        -input-
        image_id : 시각화 하기를 원하는 이미지 인덱스 번호
        figsize : 이미지 크기
        """   
        fig, ax = plt.subplots(1, figsize=figsize)
        char_boxes=boxes
        char_len=len(char_boxes)
        colors = random_colors(char_len)
        print ("box channel : ", np.shape(boxes)[-1])
        for i in range(char_len):
            color = colors[i]
        # Bounding box
            if not np.any(char_boxes[i]):
                # Skip this instance. Has no bbox. Likely lost in image cropping.
                continue
            
            if  dataset == 'icdar13' :
                if np.shape(boxes)[1] == 4 : # 4 - vertex 
                    box = char_boxes[i]
                    y_max = max(box[0, 1], box[1, 1], box[2, 1], box[3, 1])
                    y_min = min(box[0, 1], box[1, 1], box[2, 1], box[3, 1])
                    x_max = max(box[0, 0], box[1, 0], box[2, 0], box[3, 0])
                    x_min = min(box[0, 0], box[1, 0], box[2, 0], box[3, 0])

                else : # 2 - vertex 
                    box = char_boxes[i]
                    y_max = max(box[0, 1], box[1, 1])
                    y_min = min(box[0, 1], box[1, 1])
                    x_max = max(box[0, 0], box[1, 0])
                    x_min = min(box[0, 0], box[1, 0])            
                width = (x_max-x_min)
                height = (y_max-y_min)
                p = patches.Rectangle((x_min,y_min), width, height, linewidth=2,
                                      edgecolor=color, facecolor='none')
            else : 
                p = Polygon(char_boxes[i], facecolor="none", edgecolor=color)
                
            ax.add_patch(p)
        ax.imshow(image)

In [ ]:
tf.reset_default_graph()
with tf.get_default_graph().as_default():  
    with tf.Session() as sess:
        with tf.device('/cpu:0'):
            input_images = tf.placeholder(tf.float32, shape=[None, None, None, 3], name='input_images')
            global_step = tf.get_variable('global_step', [], initializer=tf.constant_initializer(0), trainable=False)

            f_score, f_geometry = model.model(input_images, is_training=False)

            variable_averages = tf.train.ExponentialMovingAverage(0.997, global_step)
            saver = tf.train.Saver(variable_averages.variables_to_restore())
            total_parameters=0
            for variable in tf.trainable_variables():  
                local_parameters=1
                shape = variable.get_shape()  #getting shape of a variable
                for i in shape:
                    local_parameters*=i.value  #mutiplying dimension values
                total_parameters+=local_parameters
            print("-----params-----" , total_parameters)   
            ckpt_state = tf.train.get_checkpoint_state(checkpoint_path)
            #model_path = os.path.join(FLAGS.checkpoint_path, os.path.basename(ckpt_state.model_checkpoint_path))
            model_path = os.path.join(checkpoint_path, os.path.basename(ckpt_state.model_checkpoint_path))
            print('Restore from {}'.format(model_path))
            saver.restore(sess, model_path)

            im_fn_list = get_images()
            im_fn_list = im_fn_list[:10]
            tot_time = time.time()
            for k,im_fn in enumerate(im_fn_list):
                im = cv2.imread(im_fn)[:, :, ::-1]
                im_resized, (ratio_h, ratio_w) = resize_image(im)
                each_time = time.time()
                timer = {'net': 0, 'restore': 0, 'nms': 0}
                start = time.time()
                score, geometry = sess.run([f_score, f_geometry], feed_dict={input_images: [im_resized]})
                timer['net'] = time.time() - start

                simple_detect_on = False 
                if simple_detect_on :
                    boxes = list(map(lambda score_map : simple_detect(score_map, np.shape(im),thres=0.3) ,[score_map for score_map in score]))
                    boxes = boxes[0] if np.shape(boxes)[0] == 1 else boxes
                else : 
                    boxes, timer = detect(score_map=score, geo_map=geometry, timer=timer)
                #print('{} : net {:.0f}ms, restore {:.0f}ms, nms {:.0f}ms'.format(
                #    im_fn, timer['net']*1000, timer['restore']*1000, timer['nms']*1000))
                #print("image shape :", np.shape(im_resized))
                #print("box shape : ", np.shape(boxes))
                #print ("each_time : ", time.time()-each_time)


                if False :
                    fig, ax = plt.subplots(7,figsize=(30,30))
                    ax[0].imshow(im_resized)
                    ax[1].imshow(score[0][:,:,0],cmap='gray')
                    ax[2].imshow(geometry[0][:,:,0])
                    ax[3].imshow(geometry[0][:,:,1])
                    ax[4].imshow(geometry[0][:,:,2])
                    ax[5].imshow(geometry[0][:,:,3])
                    ax[6].imshow(geometry[0][:,:,4])

                if simple_detect_on == True :
                    #boxes = boxes.astype(np.int32)
                    visualize(im,boxes)

                else : 
                    if boxes is not None:
                        fig, ax = plt.subplots(1,figsize=(30,30))
                        ax.imshow(score[0][:,:,0],cmap='gray')
                        boxes = boxes[:, :8].reshape((-1, 4, 2))
                        boxes[:, :, 0] /= ratio_w
                        boxes[:, :, 1] /= ratio_h
                        boxes = boxes.astype(np.int32)
                        visualize(im,boxes)


            tot_time = time.time()-tot_time
            print(tot_time)

In [ ]:
score_map = score
geo_map = geometry
print(np.shape(score),np.shape(geometry))

if len(score_map.shape) == 4:
    score_map = score_map[0, :, :, 0]
    geo_map = geo_map[0, :, :, ]
print(np.shape(geo_map),np.shape(score_map))
xy_text = np.argwhere(score_map > 0.8)
# sort the text boxes via the y axis
xy_text = xy_text[np.argsort(xy_text[:, 0])]
# restore
print(np.shape(xy_text))
start = time.time()

text_box_restored = restore_rectangle(xy_text[:, ::-1]*4, geometry[0][xy_text[:, 0], xy_text[:, 1], :]) # N*4*2
print('{} text boxes before nms'.format(text_box_restored.shape[0]))
boxes = np.zeros((text_box_restored.shape[0], 9), dtype=np.float32)
boxes[:, :8] = text_box_restored.reshape((-1, 8))
boxes[:, 8] = score_map[xy_text[:, 0], xy_text[:, 1]]
boxes = lanms.merge_quadrangle_n9(boxes.astype('float32'), 0.2)
for i, box in enumerate(boxes):
    mask = np.zeros_like(score_map, dtype=np.uint8)
    cv2.fillPoly(mask, box[:8].reshape((-1, 4, 2)).astype(np.int32) // 4, 1)
    #boxes[i, 8] = cv2.mean(score_map, mask)[0] #마스크 영역에 대해서 score 평균값 계산 . 
    print(np.sum(mask))
    print(np.sum(score_map))
    print(cv2.mean(score_map, mask))
print(boxes)

boxes = boxes[boxes[:, 8] > 0.1]
boxes = boxes[:, :8].reshape((-1, 4, 2))
boxes[:, :, 0] /= ratio_w
boxes[:, :, 1] /= ratio_h
boxes = boxes.astype(np.int32)

print("def detect, box shape : ",np.shape(boxes))
print(boxes)
visualize(im,boxes)



#timer['restore'] = time.time() - start

In [ ]:
ret, binimage = cv2.threshold(score[0],0.5,1,0)
binimage = binimage.astype(np.int8)

fig, ax  = plt.subplots(1,figsize=(10,10))
ax.imshow(binimage)
print(np.shape(score))
print(np.shape(binimage))
nlabels, labels, stats, centroids = cv2.connectedComponentsWithStats(binimage)
nlabels


In [ ]:
im_fn_list = get_images()
tot_time = time.time() 
for k,im_fn in enumerate(im_fn_list):
    read_time = time.time()
    im = cv2.imread(im_fn)[:, :, ::-1]
    read_time = time.time()-read_time
    print('{:2d}ms'.format(int(read_time*1000)),end='\r')
tot_time = time.time() - tot_time
#print (tot_time)


In [ ]:

tot_time = time.time() 
for k,im_fn in enumerate(im_fn_list):
    read_time = time.time()
    im = io.imread(im_fn)
    read_time = time.time()-read_time
    print(read_time,end='\r')
tot_time = time.time() - tot_time
print (tot_time)


In [ ]:
    im_fn_list = get_images()
        im_fn_list = im_fn_list[:2]
        t = time.time()
        for k,im_fn in enumerate(im_fn_list):
            print("Test image {:d}/{:d}".format(k+1, len(im_fn_list)), end='\r')
            im = cv2.imread(im_fn)[:, :, ::-1]
            fig , ax = plt.subplots(1,figsize=(10,10))
            ax.imshow(im)
            start_time = time.time()
            im_resized, (ratio_h, ratio_w) = resize_image(im)

            timer = {'net': 0, 'restore': 0, 'nms': 0}
            start = time.time()
            score, geometry = sess.run([f_score, f_geometry], feed_dict={input_images: [im_resized]})
            timer['net'] = time.time() - start

            boxes, timer = detect(score_map=score, geo_map=geometry, timer=timer)
            #print('{} : net {:.0f}ms, restore {:.0f}ms, nms {:.0f}ms'.format(
            #    im_fn, timer['net']*1000, timer['restore']*1000, timer['nms']*1000))

            if boxes is not None:
                boxes = boxes[:, :8].reshape((-1, 4, 2))
                boxes[:, :, 0] /= ratio_w
                boxes[:, :, 1] /= ratio_h

            duration = time.time() - start_time
            #print('[timing] {}'.format(duration))

            # save to file
            if boxes is not None:
                res_file = os.path.join(
                    output_dir,
                    'res_{}.txt'.format(
                        os.path.basename(im_fn).split('.')[0]))

                with open(res_file, 'w') as f:

                    if dataset == 'icdar15':
                        for box in boxes:
                            # to avoid submitting errors
                            box = sort_poly(box.astype(np.int32))
                            if np.linalg.norm(box[0] - box[1]) < 5 or np.linalg.norm(box[3]-box[0]) < 5:
                                continue
                            f.write('{},{},{},{},{},{},{},{}\r\n'.format(
                                box[0, 0], box[0, 1], box[1, 0], box[1, 1], box[2, 0], box[2, 1], box[3, 0], box[3, 1],
                            ))
                            cv2.polylines(im[:, :, ::-1], [box.astype(np.int32).reshape((-1, 1, 2))], True, color=(255, 255, 0), thickness=1)
                        if boxes is None : 
                            print(res_file)
                    elif dataset == 'icdar13':
                        for box in boxes:
                            box = sort_poly(box.astype(np.int32))
                            if np.linalg.norm(box[0] - box[1]) < 5 or np.linalg.norm(box[3]-box[0]) < 5:
                                continue
                            y_max = max(box[0, 1], box[1, 1], box[2, 1], box[3, 1])
                            y_min = min(box[0, 1], box[1, 1], box[2, 1], box[3, 1])
                            x_max = max(box[0, 0], box[1, 0], box[2, 0], box[3, 0])
                            x_min = min(box[0, 0], box[1, 0], box[2, 0], box[3, 0])
                            f.write('{},{},{},{} \r\n'.format(
                                x_min, y_min,x_max,y_max,
                            ))

            if not no_write_images:
                img_path = os.path.join(output_dir, os.path.basename(im_fn))
                cv2.imwrite(img_path, im[:, :, ::-1])
        print('time: ' , time.time() - t )

In [ ]:
total_parameters

In [ ]:
print(np.shape(score))
print(np.shape(geometry))

In [ ]:
import matplotlib.pyplot as plt 


In [ ]:
fig ,ax = plt.subplots(2,figsize=(10,10))
ax[0].imshow(score[0,:,:,0])
ax[1].imshow(im)
#ax[1].imshow()

In [ ]:

import icdar

In [ ]:
gen = icdar.generator(vis=True,batch_size=4)

In [ ]:
x = next(gen)

In [ ]:
x[1]
im[:, :, ::-1].astype(np.uint8)

In [ ]:
# generator return : images, image_fns, score_maps, geo_maps, training_masks
for i in range(0,4):
    print(np.shape(x))
    print(np.shape(x[0][i]))
    print(np.shape(x[3][i]))
    print(np.mean(x[0][i][:,:,:]))
    fig, ax = plt.subplots(8,figsize=(20,20))
    
    ax[0].imshow(x[0][i][:, :, ::-1].astype(np.uint8))
    ax[1].imshow(x[2][i][:,:,0])
    ax[2].imshow(x[3][i][:,:,0])
    ax[3].imshow(x[3][i][:,:,1])
    ax[4].imshow(x[3][i][:,:,2])
    ax[5].imshow(x[3][i][:,:,3])
    ax[6].imshow(x[3][i][:,:,4])      
    ax[7].imshow(x[4][i][:,:,0])
    


In [ ]:
print(np.mean(x[0][i]))
print(np.mean(x[0][i][:, :, ::-1].astype(np.uint8)))
print(np.shape(x[0][i][:, :, ::-1]))

In [ ]:
ckpt = tf.train.latest_checkpoint('./tmp/east_icdar2015_mobilenet_v2_50_rbox/')


In [ ]:
a= []
if a :
    print(a)
else :
    print("empty")

In [ ]:
print(np.newaxis)

In [ ]:
import keras.layers as KL


In [ ]:
inp = KL.Input(shape=[None,None,3])


In [ ]:
16/4

In [ ]:
resfolder = './output/res15/'
resname =[int(name.split('.')[0]) for name in os.listdir(resfolder)]
if resname : 
    resname.sort()
    print(resname)

In [ ]:
os.walk(resfolder)


In [ ]:
import sys
sys.path.append('/home/minjun/Jupyter/ocr/tf-pvanet/')
sys.path.append('/home/minjun/Jupyter/ocr/pylib/src/')
from tensorflow.contrib import slim
from pvanet import pvanet, pvanet_scope
import util
import tensorflow as tf

with slim.arg_scope(pvanet_scope(False)):
    inputs = tf.placeholder(dtype = tf.float32, shape = [None, 768, 1280, 3])
    net, end_points = pvanet(inputs,include_last_bn_relu = False,expansion=2)
    for k in sorted(end_points.keys()):
        print (k, end_points[k].shape)
    print (net.shape)

In [ ]:
from research.slim.nets.mobilenet import mobilenet_v2
inputs = tf.placeholder(dtype = tf.float32, shape = [None, 768, 1280, 3])

with slim.arg_scope(mobilenet_v2.training_scope(weight_decay=1e-5)):
    logits, endpoints = mobilenet_v2.mobilenet_base(inputs,is_training=False)

In [ ]:
endpoints['layer_18/output']


In [ ]:
import numpy as np
tuple(np.array([1,2,3]*2))

In [ ]:
"{} {}".format(1,2)

In [ ]:


num_outputs= '64 96-192 32-64-64 128 384'
num_outputs = num_outputs.split()

In [ ]:
num_outputs

In [ ]:
num_outputs = [s.split('-') for s in num_outputs]


In [ ]:
num_outputs

In [ ]:
inception_outputs = int(num_outputs[-1][0])
num_outputs = num_outputs[:-1]

In [ ]:
print(inception_outputs)
print(num_outputs)

In [ ]:
[1, 3, 3][:3]

In [ ]:
expansion = 1 if False else 2

In [ ]:
expansion

In [ ]:
fig, ax = plt.subplots(1,figsize=(30,30))
ax.imshow(im)

In [ ]:
from research.slim.nets.mobilenet.conv_blocks import expanded_conv
from research.slim.nets.mobilenet import conv_blocks as ops
from research.slim.nets.mobilenet import mobilenet as lib
expand_input = ops.expand_input_by_factor

In [ ]:
input_images = tf.placeholder(tf.float32, shape=[None, None, None, 3], name='input_images')
conv = expanded_conv(input_images,256,expansion_size= expand_input(6))

In [ ]:
from tensorflow.contrib import slim


In [ ]:
slim.model_analyzer.analyze_vars(tf.global_variables(), print_info=True)

In [ ]:
from research.slim.nets.mobilenet import mobilenet_v2
from research.slim.nets.mobilenet.conv_blocks import expanded_conv
from tensorflow.contrib import slim
input_images = tf.placeholder(tf.float32, shape=[None, None, None, 3], name='input_images')


In [ ]:
with slim.arg_scope(mobilenet_v2.training_scope(weight_decay=1e-4)):
    logits, endpoints = mobilenet_v2.mobilenet_base(input_images,is_training=False)

In [ ]:
slim.model_analyzer.analyze_vars(tf.global_variables(), print_info=True)

In [ ]:
import tensorflow as tf
ckpt_state = tf.train.get_checkpoint_state('./tmp/east_icdar2015_pvanet/')


In [ ]:
ckpt_state.all_model_checkpoint_paths[-2]